<a href="https://colab.research.google.com/github/rlatjdalsl/Phishing-Block/blob/main/stt_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔹 1. 설치 및 환경 준비
!pip install SpeechRecognition pydub
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 24.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
# 🔹 2. Google Drive 마운트 및 파일 경로 설정
from google.colab import drive
import os

drive.mount('/content/drive')
voice_file_path = '/content/drive/My Drive/Colab Notebooks/VoiceFile'
text_save_path = '/content/drive/My Drive/Colab Notebooks/TextFile'
os.makedirs(text_save_path, exist_ok=True)
file_list = os.listdir(voice_file_path)
print("📂 VoiceFile 목록:")
for file in file_list:
    print("-", file)

Mounted at /content/drive
VoiceFile 폴더 안의 파일 목록:
test.wav
model
phishing
normal
음성을 불러오는 중...
Google STT 변환 중...
인식된 텍스트: 안녕하세요 하나 둘 셋 하나 둘 셋 저는 사기꾼입니다 보이스 피싱 저는 보이스피싱범입니다 돈을 빨리 보내 주셔야 지금 아들이 저랑 같이 있어요 지금 딸이 저랑 같이 있어요 지금 어머니가 크게 다치셨어요 지금 아버지가 크게 다치셨어요 안녕하세요 검찰 지금 대포 통장이 연료 되셨는데요


In [ ]:
# 🔹 3. Google STT + 텍스트 저장 함수
import speech_recognition as sr

def google_stt_from_drive(file_name):
    recognizer = sr.Recognizer()
    file_path = os.path.join(voice_file_path, file_name)
    try:
        with sr.AudioFile(file_path) as source:
            print("🎧 음성 불러오기 중...")
            audio = recognizer.record(source)
            print("🧠 Google STT 변환 중...")
            text = recognizer.recognize_google(audio, language="ko-KR")
            print("📝 텍스트:", text)

            # 텍스트 저장
            base_name = os.path.splitext(file_name)[0] + ".txt"
            save_path = os.path.join(text_save_path, base_name)
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(text)
            print(f"✅ 텍스트 저장 완료: {save_path}")
            return text
    except sr.UnknownValueError:
        print("❌ 인식 실패")
    except sr.RequestError:
        print("❌ Google API 오류")
    return None

총 데이터 개수: 15
일반 대화: 7, 보이스피싱 대화: 8
✅ 데이터 로드 및 전처리 완료!


In [ ]:
# 🔹 4. 학습용 텍스트 데이터 로딩 및 전처리
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

normal_path = f'{voice_file_path}/normal'
phishing_path = f'{voice_file_path}/phishing'

def load_texts_from_folder(folder_path, label):
    texts, labels = [], []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as f:
                texts.append(f.read().strip())
                labels.append(label)
    return texts, labels

normal_texts, normal_labels = load_texts_from_folder(normal_path, 0)
phishing_texts, phishing_labels = load_texts_from_folder(phishing_path, 1)
texts = normal_texts + phishing_texts
labels = normal_labels + phishing_labels

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=200)
labels = np.array(labels)
print(f"✅ 총 텍스트 수: {len(texts)}")

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.8250 - loss: 0.6895
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9139 - loss: 0.6775
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.6643
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.6461
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 0.6257


✅ 모델 학습 및 저장 완료!


Mounted at /content/drive


In [ ]:
# 🔹 5. LSTM 모델 학습 및 저장
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pickle

model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    LSTM(64),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded, labels, epochs=5, batch_size=8)

model_path = f'{voice_file_path}/model'
os.makedirs(model_path, exist_ok=True)
model.save(f"{model_path}/phishing_model.h5")
with open(f"{model_path}/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ 모델 저장 완료")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔁 모델과 토크나이저 로딩 중...
✅ 모델 로딩 완료!


In [ ]:
# 🔹 6. 음성 분석 + 위험도 예측 함수
from tensorflow.keras.models import load_model

model = load_model(f"{model_path}/phishing_model.h5")
with open(f"{model_path}/tokenizer.pkl", 'rb') as f:
    tokenizer = pickle.load(f)

def analyze_audio_risk(file_name, threshold=0.5):
    text = google_stt_from_drive(file_name)
    if not text:
        return None, None
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=100)
    prediction = model.predict(padded_seq)[0][0]
    print(f"📊 위험도: {prediction:.4f}")
    print("🚨 보이스피싱 가능성 높음!" if prediction > threshold else "✅ 정상 대화")
    return text, prediction

📂 VoiceFile 폴더 안의 파일 목록:
- test.wav
- model
- phishing
- normal
🎧 음성 파일 불러오는 중: test.wav
🔁 Google STT 변환 중...
📝 인식된 텍스트: 안녕하세요 하나 둘 셋 하나 둘 셋 저는 사기꾼입니다 보이스 피싱 저는 보이스피싱범입니다 돈을 빨리 보내 주셔야 지금 아들이 저랑 같이 있어요 지금 딸이 저랑 같이 있어요 지금 어머니가 크게 다치셨어요 지금 아버지가 크게 다치셨어요 안녕하세요 검찰 지금 대포 통장이 연료 되셨는데요
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
📊 위험도 예측값: 0.0970
✅ 정상 대화로 판단됨

📊 위험도: 0.0970
✅ 위험도 낮아 LLM 분석 생략
💬 최종 판단: 정상 대화로 간주


In [ ]:
# 🔹 7. LLM 분석 함수 (OpenRouter API)
from openai import OpenAI

client = OpenAI(
    api_key="your-api-key",
    base_url="https://openrouter.ai/api/v1"
)

def llm_analysis(text, risk_score, threshold=0.5):
    if risk_score <= threshold:
        print("✅ 위험도 낮음 → LLM 생략")
        return "정상 대화로 간주"
    print("🔎 LLM 분석 시작")
    try:
        response = client.chat.completions.create(
            model="tngtech/deepseek-r1t-chimera:free",
            messages=[
                {"role": "system", "content": "당신은 보이스피싱 탐지 전문가입니다. ..."},
                {"role": "user", "content": text}
            ]
        )
        result = response.choices[0].message.content.strip()
        print("📢 LLM 분석 결과:", result)
        return result
    except Exception as e:
        print("❌ LLM 오류:", e)
        return "분석 실패"

In [ ]:
# 🔹 8. 전체 분석 실행 셀
if file_list:
    test_file = file_list[0]
    text, risk_score = analyze_audio_risk(test_file)
    if text:
        final = llm_analysis(text, risk_score)
        print("💬 최종 판단:", final)
else:
    print("❗ 분석할 파일이 없습니다.")